In [ ]:
#           AMT_INST_MIN_REGULARITY(上期信用卡應付最小金額)
#           AMT_PAYMENT_CURRENT(實際上客戶在上期信用卡還款多少)
#           AMT_INST_MIN_REGULARITY(上期貸款應付金額)

#          1.實際上還款多少>=該期貸款金額 (creditcard_high)
#          2.實際上還款多少<該期貸款金額~實際上還款多少>=應付最小金額(creditcard_normal)
#          3.實際上還款多少<應付最小金額(creditcard_low)

In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [14]:
df1= pd.read_csv("credit_card_balance.csv")


#確認負數
real_pay=df1['AMT_PAYMENT_CURRENT']
#print((real_pay < 0).sum())
should_pay=df1['AMT_TOTAL_RECEIVABLE']
#print((should_pay < 0).sum())
min_should_pay=df1['AMT_INST_MIN_REGULARITY']
#print((min_should_pay < 0).sum())

###############################################
#real_pay 遺失值約20%，10%是0，負數有0個
#10%為0不處理 我認為合裡上期沒有付款而已，沒有負數是正常值
#遺失值我想用<平均數去填>
real_pay_mean_value = real_pay.mean()
real_pay.fillna(real_pay_mean_value, inplace=True)

#檢測是否補值完

#print(real_pay.isnull().sum())
#print((real_pay==0).sum())

###############################################
#should_pay 沒有遺失值，55%是0，負數有109330個
#負數有10萬筆我視為寫錯將<負數轉為正數>
#55%是0 不太合理 應付款項不應該為0 <用平均數去補值>

#轉正數
should_pay = should_pay.abs()

#確認平均數、中位數
#print(should_pay.mean()) 
#print(should_pay.median())

#檢測補0前補0後
#print((should_pay==0).sum())
should_pay[should_pay == 0]=should_pay.mean()
#print((should_pay==0).sum())

###############################################
#min_should_pay 遺失值8%，50%是0，負數有0個
#55%是0 不太合理 最小應付款項不應該為0  <用平均數補值>
#我的做法是先用平均數補0 再去把新的資料平均數之後補遺失值

#檢測補0前補0後
#print((min_should_pay==0).sum())
min_should_pay[min_should_pay == 0]=min_should_pay.mean()
#print((min_should_pay==0).sum())

#檢測補遺失值前後
#print(min_should_pay.isnull().sum())
min_should_pay.fillna(min_should_pay.mean(), inplace=True)
#print(min_should_pay.isnull().sum())


df1['creditcard_high'] = (real_pay>=should_pay)==True
df1['creditcard_normal'] = ((real_pay<should_pay)&(real_pay>=min_should_pay))==True
df1['creditcard_low'] = (real_pay<min_should_pay)==True

#檢測單熱是否做完
"""
print(df1['creditcard_high'].head())
print('-'*80)
print(df1['creditcard_normal'].head())
print('-'*80)
print(df1['creditcard_low'].head())
print('-'*80)
normal_true = df1['creditcard_high'].sum()
print(normal_true)
print(normal_true/df1.shape[0])
nice_true = df1['creditcard_normal'].sum()
print(nice_true)
print(nice_true/df1.shape[0])
bad_true = df1['creditcard_low'].sum()
print(bad_true)
print(bad_true/df1.shape[0])
"""
def majority_vote(series):
    counts = series.value_counts()
    total_count = counts.sum()
    majority_value = counts.idxmax()
    majority_count = counts.max()
    return majority_value


result_high = df1.groupby('SK_ID_CURR').apply(lambda group: majority_vote(group['creditcard_high'])).reset_index(drop=True)
result_normal = df1.groupby('SK_ID_CURR').apply(lambda group: majority_vote(group['creditcard_normal'])).reset_index(drop=True)
result_low = df1.groupby('SK_ID_CURR').apply(lambda group: majority_vote(group['creditcard_low'])).reset_index(drop=True)

final_result = pd.DataFrame({
    'SK_ID_CURR': df1['SK_ID_CURR'].unique(),
    'creditcard_high': result_high,
    'creditcard_normal': result_normal,
    'creditcard_low': result_low
})

final_result = final_result.sort_values(by=['SK_ID_CURR'])
final_result=final_result.reset_index(drop=True)
print(final_result)

C:\Users\User\AppData\Local\Temp\ipykernel_12772\2000118743.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_should_pay[min_should_pay == 0]=min_should_pay.mean()


       SK_ID_CURR  creditcard_high  creditcard_normal  creditcard_low
89170      100006            False               True           False
27868      100011            False               True           False
13476      100013            False              False            True
89961      100021            False              False            True
95931      100023            False               True           False
...           ...              ...                ...             ...
46430      456244            False              False           False
57838      456246            False               True           False
7841       456247            False              False            True
40516      456248            False               True           False
62333      456250            False               True           False

[103558 rows x 4 columns]


In [23]:
final_result['creditcard_high'] = final_result['creditcard_high'].replace({True: 1, False: 0})
final_result['creditcard_normal'] = final_result['creditcard_normal'].replace({True: 1, False: 0})
final_result['creditcard_low'] = final_result['creditcard_low'].replace({True: 1, False: 0})

In [22]:
final_result

,SK_ID_CURR,creditcard_high,creditcard_normal,creditcard_low
0,100006,0,1,0
1,100011,0,1,0
2,100013,0,0,1
3,100021,0,0,1
4,100023,0,1,0
...,...,...,...,...
103553,456244,0,0,0
103554,456246,0,1,0
103555,456247,0,0,1
103556,456248,0,1,0
